# --- Day 19: Monster Messages ---

https://adventofcode.com/2020/day/19

This one stumped me...   

I got all the data structures set up on my own, but couldn't figure out an algorithm that would check all the messages against the decision tree of all the possible rules.   

My input data had 120 or conditions ( | ), which (to my mind) indicated that constructing all the possible strings based on the rules wouldn't be possible because 2^120 creates an ASTRONOMICALLY large number. (This, however, turns out not to be an issue, because so many of the or conditions are identical.)

In [1]:
2 ** 120

1329227995784915872903807060280344576

After reading a bunch of solutions, it appears that folks solved this problem using one of 3 techniques:

### 1) Brute force
This is what came to mind for me at first, but then I stopped because I thought that it wouldn't be possible with all the forks (or conditions). But it turns out it works.

The strategy is to collect all the possible messages in a list (or better yet, a set) based on the rules, and then loop through all the messages and see if each message is in that valid set to determine whether or not the message is valid.

**Examples:**  
https://github.com/dylan-codesYT/AdventOfCode2020/blob/master/day19.py  
https://github.com/TomBombadilV/advent-of-code-2020/blob/master/day-19/part-1.py

### 2) Recursive decent parser
When I first started poking around for clues to how to solve this puzzle, I saw folks referencing "recursive decent parser," which I'd never heard of before. So then I feel down the rabbit hole of Context Free Grammars and compiler parsers:

https://www.geeksforgeeks.org/classification-of-context-free-grammars/  
https://www.geeksforgeeks.org/recursive-descent-parser/   
https://www.geeksforgeeks.org/classification-of-top-down-parsers/  
https://www.geeksforgeeks.org/difference-between-top-down-parsing-and-bottom-up-parsing/  
https://www.booleanworld.com/building-recursive-descent-parsers-definitive-guide/  

I didn't see too many of these solutions, but I *think* this is the most obvious approach to folks who are well-versed in computer science and have maybe written a compiler parser before.

**Example:**  
https://dev.to/qviper/advent-of-code-2020-python-solution-day-19-4p9d  

### 3) Regular expresssions
This was also a fairly common approach that I read about, but haven't found too many examples for.

But I did learn a bit about Steven Cole Kleene (KLAY-nee):
https://en.wikipedia.org/wiki/Kleene_star  


**Examples:**  
https://www.youtube.com/watch?v=X-oielWtL1A



In [2]:
path = '../inputs/'

## Part 1

Part 1 requires a ["recognizer" function](https://en.wikipedia.org/wiki/Formal_grammar) to count the number of valid messages, according to a set of rule.

## Prep data

In [3]:
def get_rules_messages(filename):
    
    rules = {}
    messages = []
    
    with open(path + filename) as file:
        for line in file:
            # Parse rules into a dictionary 
            if ':' in line:
                rule_num, temp_rule = line.strip().split(': ')
                
                if '"a"' in temp_rule:
                    rule = 'a'
                elif '"b"' in temp_rule:
                    rule = 'b'
                elif '|' in temp_rule:
                    # Create list of lists when there are two options (has |)
                    rule = [list(int(n) for n in r.split(' ')) \
                            for r in temp_rule.split(' | ')]
                else:
                    # Create list of list when there is just one option (no |)
                    rule = [list(int(n) for n in temp_rule.split(' '))] 
                
                # Fill in the dictionary
                rules[int(rule_num)] = rule
            
            # Otherwise, parse the messages into a list
            elif line.startswith('a') or line.startswith('b'):
                messages.append(line.strip())
                
    return rules, messages

In [4]:
rules, messages = get_rules_messages('example_rules_messages.txt')
rules

{0: [[4, 1, 5]],
 1: [[2, 3], [3, 2]],
 2: [[4, 4], [5, 5]],
 3: [[4, 5], [5, 4]],
 4: 'a',
 5: 'b'}

## 1) Brute Force approach
Adapted from: https://github.com/TomBombadilV/advent-of-code-2020/blob/master/day-19/part-1.py  

(Most of the comments were in the original. Impressive.)

In [5]:
from typing import Dict, List, Set

def parse_rule(rules: Dict[int, int], key: int) -> Set[str]:
    """ Takes a dictionary of rules, takes one rule and makes
        all possible matches out of it into a set
    """
    # BASE CASES
    if rules[key] == 'a':
        return {'a'}
    if rules[key] == 'b':
        return {'b'}

    # RECURSIVE CASE
    else:
        # Keep track of all possible messages
        possible_messages = set()

        # For each subrule in a rule (ex: [[1, 3], [3, 1]])
        for sub_rule in rules[key]:
            # Find all possible messages for this subrule
            rule_messages = ['']

            # Parse each part separately
            for rule in sub_rule:
                # Get list of all possible messages for that part
                res = parse_rule(rules, rule)
#                 print(rule_num, sub_rule, rule, res)
                
                # Create combinations of every rule message so far
                # with every combination returned for current part
                rule_messages = [rule_message + r \
                    for rule_message in rule_messages \
                    for r in res]
#                 print(f"rule_messages: {rule_messages}")
                
            # Add messages generated by this subrule to the final message set
            # ( | is union operator on two sets)
            possible_messages = possible_messages | set(rule_messages)
            
#         print(f"possible_messages: {possible_messages} ")
        return possible_messages

In [6]:
def num_matches(rules: Dict[str, List[str]], messages: List[int], rule=0) -> int:
    """ Takes a dictionary of rules, a list of messages, and
        counts how many messages match the first rule
    """
    matches = 0
    
    # Calculate all possible messages
    possible_messages = parse_rule(rules, rule)
    print(f"There are {len(possible_messages)} possible valid messages.")

    # Check if each rule matches the indicated rule
    for message in messages:
        matches += message in possible_messages

    return matches

In [7]:
rules, messages = get_rules_messages('example_rules_messages.txt')
num_matches(rules, messages) # Should return 2

There are 8 possible valid messages.


2

In [8]:
rules, messages = get_rules_messages('rules_messages.txt')
%time num_matches(rules, messages)

There are 2097152 possible valid messages.
Wall time: 1.49 s


178

## 2) Recursive Decent Parser
Code below adapted from: https://dev.to/qviper/advent-of-code-2020-python-solution-day-19-4p9d

In [9]:
def match_rule(expression, rule_stack):
    """Recursive decsent parser that checks whether the expression is valid
        according to the production rules for the grammar.
        
    Parameters
    ----------
    expression : str
        The message (or remaining portion of the message) to be checked for validity.
    
    rule_stack : list of ints
        A list of integers, which are keys to the production rules for the grammar.
    
    Returns
    -------
    Boolean
        True if expression is valid, False otherwise.    
    """
    
    #print(expression, rule_stack)
    
    # BASE CASE
    # If there are more rules to match (len(rule_stack)) than there are characters left in
    # the message (len(expression)) then the message is too short for a full match,
    # so return False
    if len(rule_stack) > len(expression):
        return False
    
    # ANOTHER BASE CASE
    # If len(rule_stack) or len(expression) are zero, then stop recursion
    elif len(rule_stack) == 0 or len(expression) == 0:
        
        # If both are equal to zero, then function returns True because
        # the full requirements of the grammar have been met for the expression
        if len(rule_stack) == 0 and len(expression) == 0:
            return True
        
        # Otherwise, only one is equal to zero, which means the expression is 
        # either too short or too long relative to the required production rules,
        # and the function will return False.
        else:
            return False 
    
    # RECURSIVE CASE
    else:
        # Take the left-most rule off the stack (reversed rule list)
        r = rule_stack.pop()

        # if `r` is a string, that means we are at a terminus
        if isinstance(r, str):
            
            # if the terminus matches the next letter in the message (expression[0]),
            # then so far so go. 
            # Call match_rule again, but on the message advanced one position,
            # and a copy of the rule_stack.
            if expression[0] == r:
                return match_rule(expression[1:], rule_stack.copy())

            # If expression[0] != c, then the execution falls to the end of the function
            # and returns False.

        # Otherwise, we are not yet at a terminus, so add the left-most
        # rule at rule[r] to the rule_stack and call match_rule again.
        else:
            for sub_rule in rules[r]:

                # If we are at terminus, sub_rule will be either 'a' or 'b' and it will be
                # added to the right end of the rule_stack
                if match_rule(expression, rule_stack + list(reversed(sub_rule))):
                    return True

        return False

In [10]:
def count_valid_messages(rules, messages):
    num_valid_messages = 0
    for message in messages:
        if match_rule(message, list(reversed(rules[0][0]))):
            num_valid_messages += 1
    return num_valid_messages

In [11]:
rules, messages = get_rules_messages('example_rules_messages.txt')
count_valid_messages(rules, messages) # Should return 2

2

In [12]:
rules, messages = get_rules_messages('rules_messages.txt')
%time count_valid_messages(rules, messages)

Wall time: 481 ms


178

## 3) Regular Expressions

This approach produces the fastest run time and the most compact code...  
(Bypass my get_rules_messages(), and use the (very compact) parsing in compute(), below)

In [13]:
import re

In [14]:
with open(path + 'example_rules_messages.txt') as file:
    example_input = file.read()

In [15]:
with open(path + 'rules_messages.txt') as file:
    actual_input = file.read()

In [16]:
def compute(s: str) -> int:
    rules, lines = s.split('\n\n')

    rules_s = {}
    for line in rules.splitlines():
        k, _, v = line.partition(': ')  # KP: partition() is new to me...
        rules_s[k] = v

    def _get_re(s: str) -> str:
        if s == '|':
            return s

        rule_s = rules_s[s]

        # BASE CASE
        if rule_s.startswith('"'):
            return rule_s.strip('"') # Will return either 'a' or 'b'
        
        # RECURSIVE CASE
        else:
            return f'({"".join(_get_re(part) for part in rule_s.split())})'

    # Start at rule '0'
    rules_re = re.compile(_get_re('0'))
    print(rules_re)

    return sum(bool(rules_re.fullmatch(line)) for line in lines.splitlines())

In [17]:
compute(example_input) # Should return 2

re.compile('(a((aa|bb)(ab|ba)|(ab|ba)(aa|bb))b)')


2

In [18]:
%time compute(actual_input)

re.compile('((((((a(b(a(aa|ba)|b(ab))|a((bb)b|(b(b|a)|aa)a))|b((a(aa|bb)|b((b|a)(b|a)))b|((aa|bb)b|((b|a)a|ab)a)a))a|(((a(aa|ab)|b(ba))a|((bb|ba)b|(b(b|a)|aa)a)b)a|(a((ab|b(b|a))a|((b|a)(b|a))b)|b(a(ab)|b(ab|bb))
Wall time: 36 ms


178

## Part 2